# Kim Week 4 - Assignment 2

In [7]:
# imports
import sys
import gurobipy as gp
from gurobipy import GRB
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GUROBI_CMD

## Problem 1A

The dual: 

Step 1 -

Write the new objective function

Minimize $20y_{1} - 6y_{2} + 5y_{3} + 4y_{4}$

We get this by taking the RHS and plugging it into the y variables. We also have to -1 * Constraints 1 & 2 to flip the greater than equal to to less than equal to.

Step 2 - 

Write the new constraints.

We pivot to get the following (Note 3 objectives, 4 constraints becomes 4 objectives, 3 constraints):

$4y_{1} - 2y_{2} - 1y_{3} - 3y_{4} \ge -4$

$y_{1} + y_{2} + y_{3} + 2y_{4} \ge 2$

$y_{1} - 5y_{3} + y_{4} \ge 0$

and $y_{1},y_{2},y_{3},y_{4} \ge 0$








## Problem 1B

**1Bi**: The fair-market price for one unit of Resource 3 is 0 since the dual value is 0.

**1Bii**: The surplus variable value of 22.5 in the 1st dual constraint 

## Problem 2

**Part A**:

Maximize 300c + 400t - 50m1

Subject to:

$c \ge 88$ # marketing constraint for cars, at least 88

$t \ge 26$ # marketing constraint, at least 26

$2c + 3t \le 260$ # steel = 260, cars cost 2 steel and trucks cost 3 steel

$.8c + t \le m1$ #

$t1 \le 98$ # can only rent up to 98 Machine 1s

$.6c + .7t \le 73$ # Carco has 73 Machine 2s

**Part B**:

Cars to make = 88.0

Trucks to make = 27.6

Machine 1 to rent = 98.0

Objective = $32,540.00

**Part C**:

**i)** The optimal value would be the same. The allowable increase is up to a profit of $320. Since 310 is less than 320 the solution will stay the same  

**ii)** 350 is the shadow price but Carco should not be willing to pay that much. The SARHSUp column for C5, our Machine 1 Constraint, shows that the shadow price is only true up to 98.4 THe actual value added to the objective function could be less than 350 and in the worst case, less than $50 which would effectively worsen the objective value.

**iii)** For Constraint 3, our steel constraint, we see that the shadow price is 0 and the SARHSUp column is to infinity. So no matter how much steel we get, we don't get any added value. Thus Carco should not pay any additional money for steel. 

**iv)** If Carco were required to produce at least 86 cars Carco's profit would be $32,540.00 + (-20 * (88-86)) = $32,500.00. We know this from Constraint 1 being Cars >= 88, the shadow price being -20, and the lower bound for this being 85. Since 86 is within the lower bound, we can assume for decrease in cars that the shadow price will stay the same.

**v)** Carco should not produce any jeeps. By re-running the model with the jeep included, we see the objective value does not change and the amount of cars + trucks created stay the same. We can also look to the problem itself. We know that we need 88 cars and 26 trucks. If removed the 1.6 trucks made, that leaves us 1.6 days on machine 1, and 1.12 days on machine 2 free both of which are less than the 2 needed to make even 1 jeep. 

In [8]:
# NO JEEP
# variables
c = LpVariable("Cars", 0, None)
t = LpVariable("Trucks", 0, None)
m1 = LpVariable("Machine 1s", 0, None)
m2 = LpVariable("Machine 2s", 0, None)

# defines the problem
prob = LpProblem("problem", LpMaximize)

# objective function
prob += 300*c + 400*t - 50*m1

# define constraints
prob += c >= 88 # marketing constraint for cars, at least 88
prob += t >= 26 # marketing constraint, at least 26
prob += 2*c + 3*t <= 260 # cars cost 2 steel and trucks cost 3 steel with a max of 260 steel
# total number of cars made minus machine 1s used must be < or = to 0 
#(can't make more vehicles using machine 1 than the number of machine 1s)
prob += .8*c + t - m1 <= 0
prob += m1 <= 98 # can only rent up to 98 Machine 1s

prob += .6*c + .7*t <= 73 # (can't make more vehicles using machine 2 than the number of machine 2s)



# solve the problem
prob.writeLP("prob2.lp")
prob.solve(GUROBI_CMD())
print ("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob.objective))
print ("")

Status: Optimal
Cars = 88.0
Machine_1s = 98.0
Trucks = 27.599999999999994
Objective 32540.0



In [9]:
# pass the model to Gurobi
# we pass to get additional information not available in the pulp model

model = gp.read('prob2.lp')
model.optimize()

Read LP format model from file prob2.lp
Reading time = 0.00 seconds
OBJ: 6 rows, 3 columns, 10 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 9 3900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 6 rows, 3 columns and 10 nonzeros
Model fingerprint: 0x36165aaa
Coefficient statistics:
  Matrix range     [6e-01, 3e+00]
  Objective range  [5e+01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2884870e+04   5.394250e-01   0.000000e+00      0s
       2    3.2540000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.254000000e+04


In [16]:
# get the sensitivity analysis from the Gurobi model

print("Decision Variables*******")
model.printAttr(['X','OBJ','SAObjLow','SAObjUp'])

print("\nDecision Variables Additional Info (LB/UB)*******")
model.printAttr(['X','RC','LB','SALBLow','SALBUp','UB','SAUBLow','SAUBUp'])

print("\nDual Info*******")
model.printAttr(['Sense','Slack','Pi','RHS','SARHSLow','SARHSUp'])

Decision Variables*******

    Variable            X          OBJ     SAObjLow      SAObjUp 
----------------------------------------------------------------
        Cars           88          300         -inf          320 
  Machine_1s           98          -50         -400          inf 
      Trucks         27.6          400          375          inf 

Decision Variables Additional Info (LB/UB)*******

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
        Cars           88            0            0         -inf           88          inf           88          inf 
  Machine_1s           98            0            0         -inf           98          inf           98          inf 
      Trucks         27.6            0            0         -inf         27.6          inf         27.6          inf 

Du

In [11]:
# WITH JEEP
# variables
c = LpVariable("Cars", 0, None)
t = LpVariable("Trucks", 0, None)
j = LpVariable("Jeeps", 0, None)
m1 = LpVariable("Machine 1s", 0, None)
m2 = LpVariable("Machine 2s", 0, None)

# defines the probjlem
probj = LpProblem("with jeep", LpMaximize)

# objective function
probj += 300*c + 400*t + 600*j - 50*m1

# define constraints
probj += c >= 88 # marketing constraint for cars, at least 88
probj += t >= 26 # marketing constraint, at least 26
probj += j >= 0
probj += 2*c + 3*t + 4*j <= 260 # cars cost 2 steel and trucks cost 3 steel with a max of 260 steel
# total number of cars made minus machine 1s used must be < or = to 0 
#(can't make more vehicles using machine 1 than the number of machine 1s)
probj += .8*c + t + 2*j - m1 <= 0
probj += m1 <= 98 # can only rent up to 98 Machine 1s

probj += .6*c + .7*t + 2*j <= 73 # (can't make more vehicles using machine 2 than the number of machine 2s)



# solve the probjlem
probj.writeLP("probj2.lp")
probj.solve(GUROBI_CMD())
print ("Status:", LpStatus[probj.status])

for v in probj.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(probj.objective))
print ("")

Status: Optimal
Cars = 88.0
Jeeps = 0.0
Machine_1s = 98.0
Trucks = 27.599999999999994
Objective 32540.0



c:\Users\TK\Anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Question 3

Minimize Cost: $.1*(f1+f2+f3+f4) + .06*(s1+s2+s3+s4) + 0.2*(n1+n2+n3+n4)$

Where $f_{i}$ is the amount to be fast cleaned per day and $s_{i}$ is the amount to be slow cleaned per day and $n_{i}$ is the amount to be bought each day

$6.66 is the minimum cost.
For fast napkins each day: 
- Fast_Clean_FROM_Day1 = 9.0
- Fast_Clean_FROM_Day2 = 12.0
- Fast_Clean_FROM_Day3 = 6.0
- Fast_Clean_FROM_Day4 = 0.0

New Napkins each day
- New_Buy_Day1 = 15.0
- New_Buy_Day2 = 3.0
- New_Buy_Day3 = 0.0
- New_Buy_Day4 = 0.0

For the slow napklins:
- Slow_Clean_Day1_Delivered_Day3 = 6.0
- Slow_Clean_Day2_Delivered_Day4 = 0.0
- Slow_Clean_Day3_Delivered_Day5 = 0.0
- Slow_Clean_Day4_Delivered_Day6 = 0.0

In [12]:
# Question 3

# variables


f1 = LpVariable("Fast_Clean_FROM_Day1", 0, None) 
f2 = LpVariable("Fast_Clean_FROM_Day2", 0, None)
f3 = LpVariable("Fast_Clean_FROM_Day3", 0, None)
f4 = LpVariable("Fast_Clean_FROM_Day4", 0, None) # not used since no day 5?

s1 = LpVariable("Slow_Clean_Day1_Delivered_Day3", 0, None)
s2 = LpVariable("Slow_Clean_Day2_Delivered_Day4", 0, None)
s3 = LpVariable("Slow_Clean_Day3_Delivered_Day5", 0, None) # not used since no way to get it on day 4
s4 = LpVariable("Slow_Clean_Day4_Delivered_Day6", 0, None) # not used since no day 5

n1 = LpVariable("New_Buy_Day1", 0, None) 
n2 = LpVariable("New_Buy_Day2", 0, None)
n3 = LpVariable("New_Buy_Day3", 0, None)
n4 = LpVariable("New_Buy_Day4", 0, None)

# defines the problem
prob = LpProblem("problem", LpMinimize)

# objective function
prob += .1*(f1+f2+f3+f4) + .06*(s1+s2+s3+s4) + 0.2*(n1+n2+n3+n4)

# define constraints
prob += n1 == 15 # day 1 demand is 15 (assume we start from 0 napkins)

prob += f1 + n2 == 12 # day 2 demand is 12. 



prob += f2 + s1 + n3 == 18 # day 3 demand is 18 

prob += f3 + s2 + n4 == 6 # day 4 demand is 6

prob += f1 + s1 <= 15 # max clean is 15 from day 1

prob += f2 + s2 <= 12 # max clean is 15 from day 2
prob += f3  <= 18 # max clean is 15 from day 3


# solve the problem
prob.writeLP("hwprob3.lp")
prob.solve(GUROBI_CMD())
#print ("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob.objective))
print ("")

Fast_Clean_FROM_Day1 = 9.0
Fast_Clean_FROM_Day2 = 12.0
Fast_Clean_FROM_Day3 = 6.0
Fast_Clean_FROM_Day4 = 0.0
New_Buy_Day1 = 15.0
New_Buy_Day2 = 3.0
New_Buy_Day3 = 0.0
New_Buy_Day4 = 0.0
Slow_Clean_Day1_Delivered_Day3 = 6.0
Slow_Clean_Day2_Delivered_Day4 = 0.0
Slow_Clean_Day3_Delivered_Day5 = 0.0
Slow_Clean_Day4_Delivered_Day6 = 0.0
Objective 6.66



## Question 4
**Part A:**

![model](460_Prob_4.jpeg)

**Part B:**
Maximize total satisfaction:$(9*p1m_f + 8*p1fi_f + 7*p1p_f) + (9*p2m_f + 11*p2fi_f + 10*p2p_f) + (9*p3m_f + 8*p3fi_f + 10*p3p_f) +\
                            (9*p1m_s + 8*p1fi_s + 7*p1p_s) + (9*p2m_s + 11*p2fi_s + 10*p2p_s) + (9*p3m_s + 8*p3fi_s + 10*p3p_s)$

where $Pij_k$ Pi = professor number, j = course, and k = fall or spring

Professor 1 should teach:

- Prof_1_Marketing_Fall  = 3.0

- Prof_1_Marketing_Spring  = 1.0

Professor 2 should teach: 

- Prof_2_Finance_Fall  = 3.0

- Prof_2_Finance_Spring  = 1.0


Professor 3 should teach:

- Prof_3_Production_Fall  = 3.0

- Prof_3_Production_Spring  = 1.0


Total satisfaction: 120.0


In [13]:
# import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GUROBI

# variables
p1m_s = LpVariable("Prof_1_Marketing_Fall", 0, None) 
p2m_s = LpVariable("Prof_2_Marketing_Fall", 0, None)
p3m_s = LpVariable("Prof_3_Marketing_Fall", 0, None)

p1fi_s = LpVariable("Prof_1_Finance_Fall", 0, None) 
p2fi_s = LpVariable("Prof_2_Finance_Fall", 0, None)
p3fi_s = LpVariable("Prof_3_Finance_Fall", 0, None)

p1p_s = LpVariable("Prof_1_Production_Fall", 0, None) 
p2p_s = LpVariable("Prof_2_Production_Fall", 0, None)
p3p_s = LpVariable("Prof_3_Production_Fall", 0, None)

p1m_f = LpVariable("Prof_1_Marketing_Spring", 0, None) 
p2m_f = LpVariable("Prof_2_Marketing_Spring", 0, None)
p3m_f = LpVariable("Prof_3_Marketing_Spring", 0, None)

p1fi_f = LpVariable("Prof_1_Finance_Spring", 0, None) 
p2fi_f = LpVariable("Prof_2_Finance_Spring", 0, None)
p3fi_f = LpVariable("Prof_3_Finance_Spring", 0, None)

p1p_f = LpVariable("Prof_1_Production_Spring", 0, None) 
p2p_f = LpVariable("Prof_2_Production_Spring", 0, None)
p3p_f = LpVariable("Prof_3_Production_Spring", 0, None)

# defines the problem
prob = LpProblem("problem", LpMaximize)

# objective function
prob += 9*p1m_f + 8*p1fi_f + 7*p1p_f + 9*p2m_f + 11*p2fi_f + 10*p2p_f + 9*p3m_f + 8*p3fi_f + 10*p3p_f +\
    9*p1m_s + 8*p1fi_s + 7*p1p_s + 9*p2m_s + 11*p2fi_s + 10*p2p_s + 9*p3m_s + 8*p3fi_s + 10*p3p_s

# define constraints
prob += (p1m_f + p1fi_f + p1p_f) + (p1m_s + p1fi_s + p1p_s) == 4 # max of four courses for prof
prob += (p2m_f + p2fi_f + p2p_f) + (p2m_s + p2fi_s + p2p_s) == 4 # max of four courses for prof
prob += (p3m_f + p3fi_f + p3p_f) + (p3m_s + p3fi_s + p3p_s) == 4 # max of four courses for prof

prob += (p1m_f + p2m_f + p3m_f) >= 1 # at least 1 marketing section in fall
prob += (p1fi_f + p2fi_f + p3fi_f) >= 1 # at least 1 finance section in fall
prob += (p1p_f + p2p_f + p3p_f) >= 1 # at least 1 production section in fall

prob += (p1m_s + p2m_s + p3m_s) >= 1 # at least 1 marketing section in spring
prob += (p1fi_s + p2fi_s + p3fi_s) >= 1 # at least 1 finance section in spring
prob += (p1p_s + p2p_s + p3p_s) >= 1 # at least 1 production section in spring

prob += (p1m_f + p2m_f + p3m_f) + (p1m_s + p2m_s + p3m_s) <= 4 # max of 4 sections per year marketing
prob += (p1fi_f + p2fi_f + p3fi_f) + (p1fi_s + p2fi_s + p3fi_s) <= 4 # max of 4 sections per year finance
prob += (p1p_f + p2p_f + p3p_f) + (p1p_s + p2p_s + p3p_s) <= 4 # max of 4 sections per year production


# solve the problem
prob.writeLP("prob.lp")
prob.solve(GUROBI(options=['--ranges prob4v2.sen']))
#print ("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print ("Objective", value(prob.objective))
print ("")

No parameters matching 'options' found
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 9 3900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 12 rows, 18 columns and 54 nonzeros
Model fingerprint: 0x0670215e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 12 rows, 18 columns, 54 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6200000e+32   3.600000e+31   1.620000e+02      0s
       6    1.2000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.200000000e+02
Gurobi status= 2
Prof_1_Finance_Fall = 0.0
Prof_1_Finance_Spring = 0.0
Prof_1_Marketing_Fall = 3.0
Prof_1_Marketing_Spring = 1.0
Prof_1_Product